<a href="https://colab.research.google.com/github/Folio25/Projet_Seisme/blob/main/Projet_seisme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Objectif**

Réaliser une carte des villes du monde étant dans la zone d'impact de séismes ayant une amplitude supérieure à 6,1.
On considérera qu'un séisme d'amplitude 6,1 ou plus a une zone d'impact  de 100 km.
https://earthquakescanada.nrcan.gc.ca/info-gen/scales-echelles/magnitude-en.php


Le site Earthquake USGS permet de filtrer les seismes selon leur magnitude et leur dates. C'est pourquoi les données récupérées sont déjà dans le bon intervalle pour chacun des domaines.

**Liens utiles :**

[Seismes](https://earthquake.usgs.gov/earthquakes/search/)

[Villes](https://hub.arcgis.com/datasets/schools-BE::world-cities/explore)


# Installation de MapClassify

In [28]:
!pip install mapclassify

# Import des bibliothèques

In [56]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point
from google.colab import files

# Import  et traitement des données

Donnée sur les villes

In [30]:
lien_villes = "https://github.com/Folio25/Projet_Seisme/raw/refs/heads/main/Villes/World_Cities.shp"
lien_villes

'https://github.com/Folio25/Projet_Seisme/raw/refs/heads/main/Villes/World_Cities.shp'

Donnée sur les seismes

In [31]:
lien_seismes = "https://raw.githubusercontent.com/Folio25/Projet_Seisme/refs/heads/main/Seismes.csv"
lien_seismes

'https://raw.githubusercontent.com/Folio25/Projet_Seisme/refs/heads/main/Seismes.csv'

Lecture et affichage du GeoDataFrame seismes

In [32]:
gdf = gpd.read_file(lien_seismes)
gdf.head(3)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2024-12-27T12:47:37.885Z,47.3114,151.3468,146,6.8,mww,246,38,5.87,1.23,...,2025-01-28T18:14:34.040Z,Kuril Islands,earthquake,8.46,1.934,0.037,70,reviewed,us,us
1,2024-12-21T15:30:53.399Z,-17.7041,168.0033,47,6.1,mww,156,20,4.513,0.58,...,2025-01-24T15:49:05.336Z,"33 km W of Port-Vila, Vanuatu",earthquake,8.49,1.927,0.038,66,reviewed,us,us
2,2024-12-17T01:47:25.741Z,-17.6914,168.0842,54.372,7.3,mww,246,18,2.386,0.64,...,2025-01-29T22:19:43.172Z,"24 km WNW of Port-Vila, Vanuatu",earthquake,6.75,4.068,0.036,73,reviewed,us,us


Lecture et affichage du GeoDataFrame villes

In [33]:
villes = gpd.read_file(lien_villes)
villes.head(3)

,FID,OBJECTID,CITY_NAME,GMI_ADMIN,ADMIN_NAME,FIPS_CNTRY,CNTRY_NAME,STATUS,POP,POP_RANK,POP_CLASS,PORT_ID,LABEL_FLAG,POP_SOURCE,geometry
0,1,1,Cuiaba,BRA-MGR,Mato Grosso,BR,Brazil,Provincial capital,540814,3,"500,000 to 999,999",0,0,UN_Data_2010_2020,POINT (-6244244.606 -1760180.18)
1,2,2,Brasilia,BRA-DFD,Distrito Federal,BR,Brazil,National and provincial capital,2481272,2,"1,000,000 to 4,999,999",0,0,UN_Data_2010_2020,POINT (-5331952.879 -1780660.511)
2,3,3,Goiania,BRA-GOI,Goias,BR,Brazil,Provincial capital,1297154,2,"1,000,000 to 4,999,999",0,0,UN_Data_2010_2020,POINT (-5483041.697 -1889069.697)


Création d'une liste de point en se basant sur la longitude et la latitude

In [34]:
geometry = [Point(xy) for xy in zip(gdf['longitude'], gdf['latitude'])]


Rajout d'une colonne geometry et definition en WGS84

In [35]:
seismes = gpd.GeoDataFrame(gdf, geometry=geometry, crs="EPSG:4326")

Changement du système de coordonnées pour utiliser celui de OpenStreetMap

In [36]:
seismes = seismes.to_crs("EPSG:3857")

Création du buffer de 10km autout des seismes

In [37]:
buffer_seismes = seismes.buffer(100000)

Sélection des villes dans la zone du buffer

In [38]:
villes_10km = gpd.sjoin(villes, gpd.GeoDataFrame(geometry=buffer_seismes, crs=villes.crs), how="inner", predicate="intersects")

Affichage du nombre de villes concernées

In [39]:
print(len(villes_10km))

144


Permet de récupérer les noms des villes concernées

In [40]:
nom_villes = set(villes_10km['CITY_NAME'])

Permet de ne conserver que l'année pour la date

In [41]:
seismes["time"] = seismes["time"].str[:4]

Affichage de la carte

In [54]:
m = seismes.explore(tooltip = ["time", "place", "depth", "mag"], tooltip_kwds = {"aliases": ["Année :", "Lieu :", "Profondeur :", "Magnitude :"]}, popup = ["time", "place", "depth", "mag"], popup_kwds = {"aliases": ["Année :", "Lieu :", "Profondeur :", "Magnitude :"]}, name = "Seismes avec une magnitude de plus de 6,1", color = "Purple")

villes_10km.explore(m=m, tooltip = ["CITY_NAME", "POP"], tooltip_kwds = {"aliases": ["Ville :", "Population :"]}, popup = ["CITY_NAME", "POP"], popup_kwds = {"aliases": ["Ville :", "Population :"]}, name = "Villes touchées par un seisme", color = "Red")

folium.LayerControl(position='topright').add_to(m)
m

# Export de la carte en .html

In [57]:
m.save("Villes_touchées_par_un_seisme_avec_une_magnitude_de_plus_de_6,1.html")
files.download("Villes_touchées_par_un_seisme_avec_une_magnitude_de_plus_de_6,1.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>